In [1]:
#https://radimrehurek.com/gensim/models/word2vec.html
#http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/

In [1]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

C:\Users\Simo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
path = get_tmpfile("word2vec.model")

In [3]:
model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)

In [4]:
model.save("word2vec.model")

In [5]:
model = Word2Vec.load("word2vec.model")

In [6]:
model.train([["hello", "world"]], total_examples=1, epochs=1)

(0, 2)

In [7]:
vector = model.wv['computer']

In [8]:
from gensim.models import KeyedVectors

In [9]:
path = get_tmpfile("wordvectors.kv")

In [10]:
model.wv.save(path)

In [11]:
wv = KeyedVectors.load("model.wv", mmap='r')

FileNotFoundError: [Errno 2] No such file or directory: 'model.wv'